# Load and Switch PEFT LoRA Adapters on SLMs

Here we will use the two LoRA adapters we built previously and show you how you can efforlessly switch between different NLP tasks by keeping the same base SLM and just switching your LoRA task-based adapters

In [2]:
from transformers import (AutoModelForTokenClassification, 
                          AutoModelForSequenceClassification,
                          AutoTokenizer,
                          pipeline)

## Load Classification LoRA Adapter on Base BERT

In [3]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

cls_model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-uncased',
                                                                id2label=id2label,
                                                                label2id=label2id,
                                                                num_labels=2)
cls_tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased', 
                                          fast=True)

cls_model.load_adapter(peft_model_id='qlora-distilbert-sentiment-adapter',
                       adapter_name='sentiment-classifier')
cls_model = cls_model.eval()
cls_model = cls_model.to('cuda')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
cls_pipe = pipeline(task='text-classification', 
               model=cls_model, tokenizer=cls_tokenizer, 
               device='cuda')

## Load NER LoRA Adapter on Base BERT

In [6]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

ner_model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", 
    num_labels=13, 
    id2label=id2label, 
    label2id=label2id,
)
ner_tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased', 
                                          fast=True)

ner_model.load_adapter(peft_model_id='qlora-distilbert-ner-adapter',
                       adapter_name='ner')
ner_model = ner_model.eval()
ner_model = ner_model.to('cuda')

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
ner_pipe = pipeline(task='ner', 
               model=ner_model, tokenizer=ner_tokenizer, 
               device='cuda')

## Test same model for different tasks based on the different loaded adapters above

In [8]:
cls_pipe('This is definitely not a good movie')

[{'label': 'NEGATIVE', 'score': 0.9594546556472778}]

In [9]:
ner_pipe('Google Search was invented by Larry Page and Sergey Brin')

[{'entity': 'B-corporation',
  'score': 0.7544448,
  'index': 1,
  'word': 'google',
  'start': 0,
  'end': 6},
 {'entity': 'B-person',
  'score': 0.92128485,
  'index': 6,
  'word': 'larry',
  'start': 30,
  'end': 35},
 {'entity': 'I-person',
  'score': 0.9446377,
  'index': 7,
  'word': 'page',
  'start': 36,
  'end': 40},
 {'entity': 'B-person',
  'score': 0.9292107,
  'index': 9,
  'word': 'sergey',
  'start': 45,
  'end': 51},
 {'entity': 'I-person',
  'score': 0.90647864,
  'index': 10,
  'word': 'br',
  'start': 52,
  'end': 54},
 {'entity': 'I-person',
  'score': 0.49488345,
  'index': 11,
  'word': '##in',
  'start': 54,
  'end': 56}]